In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams.update({'font.size': 18})

### Part - a

The general expression of paraboloid (assuming circular symmetry) can be easily modeled as follows
Ideally z = a(x^2+y^2) where the center is at origin, but this might not be the case always, rather we will have some offset of (x0,y0,z0).
So, $z-z0 = a((x-x0)^2+(y-y0)^2)$

Then 
$$z - z0 = a(x^2 + y+^2 - 2x0*x - 2y0*y + (x0^2 + y0^2))$$ 
Defining - 
$$r^2 = x^2 + y^2$$
$$z = (z0 + a(x0^2 + y0^2)) - 2ax0*x - 2ay0*y + ar^2$$ 
for R = r^2, we can define z in terms of linear parameters

Finally:
***
$$z = (z0 + a(x0^2 + y0^2)) - 2ax0*x - 2ay0*y + aR$$
***

Starting of with reading from data-file and structuring data in easily usable form:

In [3]:
f = open('dish_zenith.txt','r')
xyz = []
for t in f:
	t = np.asarray(t.strip().split(), dtype = 'float')
	xyz.append(t)
f.close()	

xyz = np.asarray(xyz)	
x = xyz[:,0]
y = xyz[:,1]
z = xyz[:,2]
R = x**2+y**2

Defining the coefficient matrix, A and calculating the coefficients:

In [4]:
A = np.transpose([np.ones(len(z)),x,y,R])

The form we are aiming for here is z = A@c, where

$$c[0] = z0 + a(x0^2 + y0^2)$$
$$c[1] = -2ax0$$
$$c[2] = -2ay0$$
$$c[3] = a$$

### Part - b (best-fit parameters)

In [5]:
# Performing the fit
coeff = np.linalg.lstsq(A,z,rcond=None)[0]
# Comparing with our above definintion of the parameters
a = coeff[3]
y0 = -0.5*coeff[2]/a
x0 = -0.5*coeff[1]/a
z0 = coeff[0] - a*(x0**2+y0**2)

f = 1/(4*a)
par = [x0,y0,z0,a]
print('\nBest fit parameters:')
print('{x0,y0,z0,a} = ',par)


Best fit parameters:
{x0,y0,z0,a} =  [-1.3604886221977073, 58.22147608157965, -1512.8772100367876, 0.00016670445477401328]


***
### Part - c

Simple method to quantify the error/noise I employ here is to calculate the the error in the data, i.e. the standard deviation in z

In [6]:
stdz = abs(np.std(z-(a*((x-x0)**2+(y-y0)**2)+z0)))
print('Std(z) = ',stdz)

Std(z) =  3.768338648784738


With the std in z known we can relate it to 'a' as:
$std(z) / z_0=std(a) / a$

'a' in turn relates to f as: $a=1/(4f)$

In [7]:
stda = abs(stdz*a/z0)
stdf = stda*f/a
print('Value of focal length(mm): ',[f])
print('The error is well withing 1sigma of actual value(1500mm), error = ',abs(f-1500),' < ',stdf)

Value of focal length(mm):  [1499.6599841252187]
The error is well withing 1sigma of actual value(1500mm), error =  0.34001587478132933  <  3.735416622527846


### Bonus

Here is my take on the extension of current question for the case when we consider dish to me not circularly symmetric. This would require us to fit data to:
$$z = a*x'^2 + b*y'^2$$ 
The above results give a good enough value for x0,y0,z0 but fails to find the direction of the primary axis and it cant be assumed the the provided (x,y) are along those axes. So for this case
$$x = cos(t)*x'+sin(t)*y'$$
$$y = cos(t)*y'-sin(t)*x'$$
Hence we need to inverse rotate since we have x,y instead of actual ones
$$x' = cos(t)*x-sin(t)*y$$
$$y' = cos(t)*y+sin(t)*x$$
This gives:
$$z = a(cos(t)*x-sin(t)*y)^2 + b(cos(t)*y+sin(t)*x)^2$$
$$z = (ac^2+bs^2)*x^2+(as^2+bc^2)*y^2+2*(b-a)cs*(xy)$$
$$z = (ac^2+bs^2)*X+(as^2+bc^2)*Y+2*(b-a)cs*W$$
where,
$$X = x^2, Y = y^2, W = xy$$


In [8]:
# First taking care of offsets using earlier results:
x = x-x0
y = y-y0
z = z-z0
# Redefining terms:
X = x**2
Y = y**2
W = x*y

A = np.transpose([X,Y,W])
coeff2 = np.linalg.lstsq(A,z,rcond=None)[0]

The 3 coefficient obtained are thus used to calculate the angle the current coordinate system makes with previous one, and also the get (a,b) which directly relate to the focal lenghts along the twp principal axes. (Work-up in included separately in the same folder).

In [9]:
a1 = coeff2[0]
a2 = coeff2[1]
a3 = coeff2[2]

$$\Theta = 0.5*tan^{-1}(a_3/(a_2-a_1))$$
$$a = 0.5(a_1+a_2-a_3/(2sin(\Theta)cos(\Theta)))$$
$$a = 0.5(a_1+a_2+a_3/(2sin(\Theta)cos(\Theta)))$$

In [10]:
theta = 0.5*np.arctan(a3/(a2-a1))
a_new = 0.5*(a1+a2-a3/(2*np.cos(theta)*np.sin(theta)))
b_new = 0.5*(a1+a2+a3/(2*np.cos(theta)*np.sin(theta)))
print('\nAngle(degrees): ',theta*180/np.pi)
print('New focal lengths: f_a=',1/(4*a_new),' f_b=',1/(4*b_new))



Angle(degrees):  35.537288720377
New focal lengths: f_a= 1508.592256467945  f_b= 1490.3685134380532


(all units in mm, degrees) 
***
Thus we get the focal lengths along the 2 axes as: **(1508.59,1490.37)**